# Imports and Preprocessing

In [27]:
import numpy as np
import pandas as pd
import tensorflow as tf
from scikeras.wrappers import KerasRegressor

In [28]:
import os
import pandas as pd
folder_path = 'Data_Test_v1'
file_names = ['data_test.csv', 'data_train.csv', 'target_test.csv', 'target_train.csv']

data_frames = []
for file_name in file_names:
    file_path = os.path.join(folder_path, file_name)
    df = pd.read_csv(file_path)
    data_frames.append(df)

data_test = data_frames[0]
data_train = data_frames[1]
target_test = data_frames[2]    
target_train = data_frames[3]

# ANN Model

In [29]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from scikeras.wrappers import KerasRegressor
from sklearn.model_selection import RandomizedSearchCV
from tensorflow.keras.optimizers import Adam
from scipy.stats import reciprocal

# Define function to create the ANN model 
def create_model(n_hidden=1, n_neurons=30, input_shape=[3]): #learning_rate_input=3e-3, 
    model = Sequential()
    model.add(Dense(n_neurons, activation="relu", input_shape=input_shape))
    for layer in range(n_hidden):
        model.add(Dense(n_neurons, activation="relu"))
    model.add(Dense(1))
    optimizer = Adam() #learning_rate=learning_rate_input
    model.compile(loss="mean_squared_error", optimizer=optimizer)
    return model

# Define function to build the model
def model_builder(n_hidden=1, n_neurons=30, ): #learning_rate=3e-3
    return KerasRegressor(build_fn=create_model, verbose=0, n_hidden=n_hidden, n_neurons=n_neurons) #, learning_rate=learning_rate

# Define the hyperparameters to search for the Randomized Search
param_distribs = {
    "n_hidden": [0, 1, 2, 3],
    "n_neurons": np.arange(1, 100),
    #"learning_rate": reciprocal(1e-4, 1e-2),
    "batch_size": [16, 32, 64, 128],
    "epochs": [10, 20, 30],
    #"activation": ['relu', 'sigmoid', 'tanh']
}

# Create the model and optimize with Randomized Search
model_ANN = model_builder()
rnd_search_cv_ANN = RandomizedSearchCV(model_ANN, param_distribs, n_iter=10, cv=3)

# Fit the model on the training data
rnd_search_cv_ANN.fit(data_train, target_train)

# Predict the target values
target_pred_ANN = rnd_search_cv_ANN.predict(data_test)

c:\Users\lanza\Integrated-vs-Seperated-Master-Thesis\.venv\Lib\site-packages\scikeras\wrappers.py:915: UserWarning: ``build_fn`` will be renamed to ``model`` in a future release, at which point use of ``build_fn`` will raise an Error instead.
  X, y = self._initialize(X, y)
c:\Users\lanza\Integrated-vs-Seperated-Master-Thesis\.venv\Lib\site-packages\scikeras\wrappers.py:915: UserWarning: ``build_fn`` will be renamed to ``model`` in a future release, at which point use of ``build_fn`` will raise an Error instead.
  X, y = self._initialize(X, y)
c:\Users\lanza\Integrated-vs-Seperated-Master-Thesis\.venv\Lib\site-packages\scikeras\wrappers.py:915: UserWarning: ``build_fn`` will be renamed to ``model`` in a future release, at which point use of ``build_fn`` will raise an Error instead.
  X, y = self._initialize(X, y)
c:\Users\lanza\Integrated-vs-Seperated-Master-Thesis\.venv\Lib\site-packages\scikeras\wrappers.py:915: UserWarning: ``build_fn`` will be renamed to ``model`` in a future relea

# DT using LightGBM

In [30]:
import lightgbm as lgb

# Create the model
model_DT = lgb.LGBMRegressor()

# Define the hyperparameters to search for the Randomized Search
param_distribs = {
    "n_estimators": [50, 100, 200],
    "max_depth": [3, 5, 7],
    "learning_rate": [0.01, 0.1, 0.5],
    "subsample": [0.8, 1.0],
    "colsample_bytree": [0.8, 1.0],
}

# Create the model and optimize with Randomized Search
rnd_search_cv_DT = RandomizedSearchCV(model_DT, param_distribs, n_iter=10, cv=3)

# Fit the model on the training data
rnd_search_cv_DT.fit(data_train, target_train)

# Make predictions
target_pred_DT = rnd_search_cv_DT.predict(data_test)

[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000020 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 49
[LightGBM] [Info] Number of data points in the train set: 138, number of used features: 3
[LightGBM] [Info] Start training from score 1561.014493
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, b

# Forecasting Error

In [32]:
# Calculate forecast error for ANN
train_pred_ANN = rnd_search_cv_ANN.predict(data_train)
train_pred_ANN_flatten = train_pred_ANN.flatten()
target_train = target_train.values.flatten()
forecast_error_ANN = target_train - train_pred_ANN_flatten

In [34]:
# Calculate forecast error for DT
train_pred_DT = rnd_search_cv_DT.predict(data_train)
train_pred_DT_flatten = train_pred_DT.flatten()
forecast_error_DT = target_train - train_pred_DT_flatten

[LightGBM] [Warning] Accuracy may be bad since you didn't explicitly set num_leaves OR 2^max_depth > num_leaves. (num_leaves=31).


# Predicting Order Quantities 

In [35]:
# Initialize an empty list to store the final order quantities
final_order_quantities_ANN = []
final_order_quantities_DT = []

# Define costs
c = 2  # cost per unit of product
s = 1  # salvage value per unit of leftover product
p = 3  # price per unit of product sold

# Calculate critical ratio
critical_ratio = (p - c) / ((p - s) + (p - c))

In [36]:
from scipy.stats import norm
# Loop over each week in data_test
for i in range(len(data_test)):

    # Number of scenarios
    num_scenarios = 1000

    # Generate demand scenarios for the week
    demand_scenarios_single = target_pred_ANN[i] + np.random.choice(forecast_error_ANN, size=num_scenarios)

    # Solve newsvendor problem for the week
    order_quantity_scenarios = norm.ppf(critical_ratio, loc=demand_scenarios_single)

    # Append the order quantity to the list
    final_order_quantities_ANN.append(np.mean(order_quantity_scenarios))

# Convert the list to a numpy array
final_order_quantities_ANN = np.array(final_order_quantities_ANN)

In [37]:
# Loop over each week in data_test
for i in range(len(data_test)):

    # Number of scenarios
    num_scenarios = 1000

    # Generate demand scenarios for the week
    demand_scenarios_single = target_pred_DT[i] + np.random.choice(forecast_error_DT, size=num_scenarios)

    # Solve newsvendor problem for the week
    order_quantity_scenarios = norm.ppf(critical_ratio, loc=demand_scenarios_single)

    # Append the order quantity to the list
    final_order_quantities_DT.append(np.mean(order_quantity_scenarios))

# Convert the list to a numpy array
final_order_quantities_DT = np.array(final_order_quantities_DT)

In [38]:
print(final_order_quantities_ANN)

[1516.17212487 1526.04988391 1540.48655761 1535.20914733 1535.04268994
 1530.81900244 1528.7513894  1537.0364165  1528.21107055 1549.12600366
 1522.8400758  1523.40400915 1541.23524682 1543.29437866 1547.65140588
 1532.92661096 1538.00274035 1533.1998833  1539.78828027 1533.17738513
 1531.47790405 1515.50859753 1532.58215686 1538.31624707 1539.05138171
 1530.24605346 1522.21020288 1545.72299243 1523.33838903 1538.01558215
 1532.65941601 1541.90773571 1536.34005615 1552.53281481 1529.21847009
 1517.70379187 1508.03389196 1534.70683532 1545.36289331 1540.1284624
 1516.68195532 1542.54806018 1538.37640307 1535.88349694 1532.26638012
 1541.02527221 1533.55037133 1535.29583544 1536.196557   1528.46718518
 1543.22842785 1521.24068408 1526.36389453]


# Calculate Costs

In [39]:
# Loop over each week in target_test
overall_profit_ANN = 0
overall_profit_DT = 0
for i in range(len(target_test)):

    # Calculate the profit for the week (without over or understock costs)
    profit_ANN = (p - c) * round(final_order_quantities_ANN[i])
    profit_DT = (p - c) * round(final_order_quantities_DT[i])

    # Calculate understock and overstock costs
    overstock_ANN = 0
    overstock_DT = 0
    understock_ANN = 0
    understock_DT = 0
    if final_order_quantities_ANN[i] < target_test.values[i]:
        understock_ANN = (p - c) * (target_test.values[i] - round(final_order_quantities_ANN[i]))
    if final_order_quantities_ANN[i] > target_test.values[i]:
        overstock_ANN = (c - s) * (round(final_order_quantities_ANN[i]) - target_test.values[i])
    if final_order_quantities_DT[i] < target_test.values[i]:
        understock_DT = (p - c) * (target_test.values[i] - round(final_order_quantities_DT[i]))
    if final_order_quantities_DT[i] > target_test.values[i]:
        overstock_DT = (c - s) * (round(final_order_quantities_DT[i]) - target_test.values[i])
    

    # Calculate the total profit for the week
    total_profit_ANN = profit_ANN - overstock_ANN - understock_ANN
    total_profit_DT = profit_DT - overstock_DT - understock_DT

    # Add the total profit to the overall profit
    overall_profit_ANN += total_profit_ANN
    overall_profit_DT += total_profit_DT
   

# Print the overall profit
print('Overall profit for ANN: ', overall_profit_ANN)
print('Overstock: ', overstock_ANN, ' Understock: ', understock_ANN)
print('Overall profit for DT: ', overall_profit_DT)
print('Overstock: ', overstock_DT, ' Understock: ', understock_DT)

Overall profit for ANN:  [65599]
Overstock:  0  Understock:  [325]
Overall profit for DT:  [78773]
Overstock:  [9]  Understock:  0
